In [1]:
%matplotlib inline
import pywt
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.colors as colors
from mpl_toolkits.axes_grid1 import make_axes_locatable
import seaborn as sns


from pandas import set_option
set_option("display.max_rows", 10)
pd.options.mode.chained_assignment = None

filename = 'datafacies_vectors.csv'
training_data = pd.read_csv(filename)
training_data

OSError: File b'facies_vectors.csv' does not exist

In [ ]:
db3 = pywt.Wavelet('db3')

In [ ]:
grouped = training_data.groupby(['Well Name'])
max_GR_levels = []
max_ILD_levels = []
max_DeltaPHI_levels = []
max_PHIND_levels = []
max_PE_levels = []


for key in grouped.groups.keys():
    max_GR_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).GR),db3))
    max_ILD_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).ILD_log10),db3))
    max_DeltaPHI_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).DeltaPHI),db3))
    max_PHIND_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).PHIND),db3))
    max_PE_levels.append(pywt.dwt_max_level(len(grouped.get_group(key).PE),db3))

print(max_GR_levels,max_ILD_levels,max_DeltaPHI_levels,max_PHIND_levels,max_PE_levels)

In [ ]:
from scipy.interpolate import interp1d

In [ ]:
def restore_df(group):
    return group.reset_index(drop=True)

In [ ]:
n_levels = 6
new_vars = pd.DataFrame()

for key in grouped.groups.keys():
    
    depth = grouped.get_group(key)['Depth']
    temp_df = pd.DataFrame()
    temp_df['Depth'] = depth
    for log in ['GR','ILD_log10','DeltaPHI','PHIND','PE']:
      
        temp_data = grouped.get_group(key)[log]
        
        for i in range(1,n_levels+1):

            cA,cD = pywt.dwt(temp_data,db3,mode='symmetric')

            new_depth = np.linspace(min(depth),max(depth),len(cD))
            f1 = interp1d(new_depth,cD,kind='linear')
            f2 = interp1d(new_depth,cD,kind='nearest')
            f3 = interp1d(new_depth,cA,kind='linear')

#             temp_df[log + '_cD_lin_level_' + str(i)] = f1(depth)
            temp_df[log + '_cD_step_level_' + str(i)] = f2(depth)
#             temp_df[log + '_cA_level_' + str(i)] = f3(depth)
    
    temp_df['Well Name'] = [key for _ in range(len(depth))]
    new_vars = new_vars.append(temp_df)
                

In [ ]:
new_vars = new_vars.sort_index()

In [ ]:
new_vars.to_pickle('data/vars_from_dwt.pkl')

### Plot the result